In [1]:
!pip install kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"meesalasaidhanush","key":"3128d88657acdfedd8f3215166565e00"}'}

In [3]:
!mkdir -p ~/.kaggle

In [4]:
!mv kaggle.json ~/.kaggle/


In [5]:
!chmod 600 ~/.kaggle/kaggle.json


In [7]:
!kaggle datasets download -d mohamedbakhet/amazon-books-reviews

 99% 1.05G/1.06G [00:11<00:00, 188MB/s]
100% 1.06G/1.06G [00:11<00:00, 100MB/s]


In [8]:
!unzip /content/amazon-books-reviews.zip

Archive:  /content/amazon-books-reviews.zip
  inflating: Books_rating.csv        
  inflating: books_data.csv          


In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("/content/Books_rating.csv")

In [3]:
df.head(1)

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...


In [4]:
df.drop(columns=["Id","Title","Price","User_id","profileName","review/helpfulness","review/time","review/text"],inplace=True)

In [5]:
df.shape

(3000000, 2)

In [14]:
df["review/score"].value_counts()

5.0    1807343
4.0     585616
3.0     254295
1.0     201688
2.0     151058
Name: review/score, dtype: int64

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import naive_bayes
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
%matplotlib inline

In [18]:
stopset=set(stopwords.words('english'))
vector=TfidfVectorizer(use_idf=True,lowercase=False,strip_accents='ascii',stop_words=stopset)

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
pip install cleantext

In [7]:
import cleantext


In [31]:
cleantext.clean("Dhah 44 ' '/ fhecon" )


'dhah fhecon'

In [32]:
def f(s):
  sg=cleantext.clean(s,
  lowercase=True ,# Convert to lowercase
  numbers=True ,# Remove all digits
  )
  return sg

In [38]:
df.head()

,review/score,review/summary
0,4.0,Nice collection of Julie Strain images
1,5.0,Really Enjoyed It
2,5.0,Essential for every personal and Public Library
3,4.0,Phlip Nel gives silly Seuss a serious treatment
4,4.0,Good academic overview


In [39]:
print(type(df["review/summary"][0]))

<class 'str'>


In [9]:
def strip_if_string(value):
    if isinstance(value, str):
        return value.strip()
    elif isinstance(value, float):
        return str(value).strip()  # Convert float to string and then strip
    else:
        return value

In [10]:
df["review/summary"]=df["review/summary"].apply(strip_if_string)

In [33]:
subset_df = df.iloc[:100000]

In [18]:
subset_df.head()

,review/score,review/summary
0,4.0,Nice collection of Julie Strain images
1,5.0,Really Enjoyed It
2,5.0,Essential for every personal and Public Library
3,4.0,Phlip Nel gives silly Seuss a serious treatment
4,4.0,Good academic overview


In [14]:
subset_df["review/score"].value_counts()

5.0    60212
4.0    19222
3.0     8334
1.0     6985
2.0     5247
Name: review/score, dtype: int64

In [34]:
subset_df["review/summary"]=subset_df["review/summary"].apply(f)

<ipython-input-34-1ce4cb349562>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df["review/summary"]=subset_df["review/summary"].apply(f)


In [35]:
subset_df.head()

,review/score,review/summary
0,4.0,nice collection of julie strain images
1,5.0,really enjoyed it
2,5.0,essential for every personal and public library
3,4.0,phlip nel gives silly seuss a serious treatment
4,4.0,good academic overview


In [37]:
subset_df.to_csv("sentiment.csv",index=False)

In [38]:
from imblearn.over_sampling import SMOTE


In [41]:
x=subset_df['review/summary']
y=subset_df['review/score']

In [42]:
print('Before OverSampling, counts of label 1: {}'.format(sum(y==1.0)))
print('Before OverSampling, counts of label 2: {} \n'.format(sum(y==2.0)))
print('Before OverSampling, counts of label 3: {}'.format(sum(y==3.0)))
print('Before OverSampling, counts of label 4: {} \n'.format(sum(y==4.0)))
print('Before OverSampling, counts of label 5: {}'.format(sum(y==5.0)))

Before OverSampling, counts of label 1: 6985
Before OverSampling, counts of label 2: 5247 

Before OverSampling, counts of label 3: 8334
Before OverSampling, counts of label 4: 19222 

Before OverSampling, counts of label 5: 60212


In [43]:

from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()

x=cv.fit_transform(x)

In [44]:
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(x,y.ravel())

In [45]:

print('After OverSampling, counts of label 1: {}'.format(sum(y_train_res == 1.0)))
print('After OverSampling, counts of label 2: {}'.format(sum(y_train_res == 2.0)))
print('After OverSampling, counts of label 3: {}'.format(sum(y_train_res == 3.0)))
print('After OverSampling, counts of label 4: {}'.format(sum(y_train_res == 4.0)))
print('After OverSampling, counts of label 5: {}'.format(sum(y_train_res == 5.0)))

After OverSampling, counts of label 1: 60212
After OverSampling, counts of label 2: 60212
After OverSampling, counts of label 3: 60212
After OverSampling, counts of label 4: 60212
After OverSampling, counts of label 5: 60212


In [51]:
import pickle

from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
x=subset_df['review/summary']
y=subset_df['review/score']
x=cv.fit_transform(x)

pickle.dump(cv,open('trans.pkl','wb'))



from sklearn.tree import DecisionTreeClassifier
clf=DecisionTreeClassifier()
clf.fit(X_train_res,y_train_res)


pickle.dump(clf,open('nlp_m21.pkl','wb'))

KeyboardInterrupt: ignored

In [52]:
import sklearn

In [53]:
sklearn. __version__

'1.2.2'